In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer

from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.util import ngrams

import string
import re

In [2]:
df = pd.read_csv("C:/Users/manth/Project/Text Classification for Hotel Review/train (2).csv")
df.head()

ID                                             Review  Rating
0   0  exceptional service nice all-around daughter s...       5
1   1  beautiful relaxing jw marriott desert ridge re...       5
2   2  great location great location 5 mins subway ta...       5
3   3  pleased nice safe hotel, flower market hotel v...       3
4   4  excellent hotel service great hotel excellent ...       4

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14343 entries, 0 to 14342
Data columns (total 3 columns):
ID        14343 non-null int64
Review    14343 non-null object
Rating    14343 non-null int64
dtypes: int64(2), object(1)
memory usage: 336.3+ KB


In [4]:
df.isnull().sum()

ID        0
Review    0
Rating    0
dtype: int64

In [5]:
print(df.Rating.value_counts())

5    6436
4    4172
3    1510
2    1248
1     977
Name: Rating, dtype: int64


In [6]:
df.Rating.describe()

count    14343.000000
mean         3.965070
std          1.230439
min          1.000000
25%          3.000000
50%          4.000000
75%          5.000000
max          5.000000
Name: Rating, dtype: float64

In [7]:
df.groupby('Rating').describe()

ID                                                             \
         count         mean          std   min      25%     50%       75%   
Rating                                                                      
1        977.0  7138.492323  4130.144108  32.0  3577.00  6917.0  10751.00   
2       1248.0  7089.788462  4150.087921  13.0  3466.50  7050.5  10632.00   
3       1510.0  7101.452318  4114.071624   3.0  3497.25  7286.0  10661.25   
4       4172.0  7251.357622  4141.163144   4.0  3686.50  7296.5  10911.50   
5       6436.0  7155.909416  4146.643874   0.0  3566.75  7136.5  10708.25   

                 
            max  
Rating           
1       14342.0  
2       14330.0  
3       14341.0  
4       14340.0  
5       14339.0

In [8]:
# text preprocessing

def preprocessing (document):
    
    document = str(document).lower() # convert to lower cases
    document = re.sub(r'\d+', '', document) # remove numbers
    document = document.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
    document = document.strip() # remove white space
    
    tokenizer = RegexpTokenizer(r'\w+') # tokenize
    tokens = tokenizer.tokenize(document)
    
    # stop words
    stop_words = stopwords.words('english')  
    filtered_words = [w for w in tokens if not w in stop_words] # remove stop words
        
    # generate bigrams
    bigrams = ngrams(tokens, 2)
    # convert to list of bigram lists
    bigrams_clean = [list(generator) for generator in bigrams]
    # combine stopword list
    stop_words = stopwords.words('english')  
   
    # remove bigram if BOTH words are in stopword list
    bigrams_clean = [bigram for bigram in bigrams_clean if not all(word in stop_words for word in bigram)]
    # stem bigrams
    wordnet_lemmatizer = WordNetLemmatizer()
    bigrams_clean = [(wordnet_lemmatizer.lemmatize(bigram[0]),wordnet_lemmatizer.lemmatize(bigram[1])) for bigram in bigrams_clean]
        
        
        # stemmer
        #stemmer = EnglishStemmer()
        #filtered_words_stem = [stemmer.stem(w) for w in filtered_words]
        
    # Lemmatization 
    wordnet_lemmatizer = WordNetLemmatizer()
    filtered_words_lem = [wordnet_lemmatizer.lemmatize(w) for w in filtered_words]
                
    return (filtered_words_lem, bigrams_clean)

In [9]:
results = [preprocessing(reviews) for reviews in df['Review']]
cleaned_df = pd.DataFrame(results, columns=['unigram_clean', 'bigrams_clean'])

In [10]:
df1 = pd.concat([df, cleaned_df], axis=1)
df1.head()

ID                                             Review  Rating  \
0   0  exceptional service nice all-around daughter s...       5   
1   1  beautiful relaxing jw marriott desert ridge re...       5   
2   2  great location great location 5 mins subway ta...       5   
3   3  pleased nice safe hotel, flower market hotel v...       3   
4   4  excellent hotel service great hotel excellent ...       4   

                                       unigram_clean  \
0  [exceptional, service, nice, allaround, daught...   
1  [beautiful, relaxing, jw, marriott, desert, ri...   
2  [great, location, great, location, min, subway...   
3  [pleased, nice, safe, hotel, flower, market, h...   
4  [excellent, hotel, service, great, hotel, exce...   

                                       bigrams_clean  
0  [(exceptional, service), (service, nice), (nic...  
1  [(beautiful, relaxing), (relaxing, jw), (jw, m...  
2  [(great, location), (location, great), (great,...  
3  [(pleased, nice), (nice, safe), (safe, hotel),...  
4  [(excellent, hotel), (hotel, service), (servic...

In [11]:
# return the wordnet object value corresponding to the POS tag
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    

In [12]:
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
     # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

# clean text data
df["review_clean"] = df["Review"].apply(lambda x: clean_text(x))

In [13]:
df.head()

ID                                             Review  Rating  \
0   0  exceptional service nice all-around daughter s...       5   
1   1  beautiful relaxing jw marriott desert ridge re...       5   
2   2  great location great location 5 mins subway ta...       5   
3   3  pleased nice safe hotel, flower market hotel v...       3   
4   4  excellent hotel service great hotel excellent ...       4   

                                        review_clean  
0  exceptional service nice all-around daughter s...  
1  beautiful relax jw marriott desert ridge resor...  
2  great location great location mins subway take...  
3  pleased nice safe hotel flower market hotel va...  
4  excellent hotel service great hotel excellent ...

In [14]:

# add sentiment anaylsis columns
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
df["sentiments"] = df["Review"].apply(lambda x: sid.polarity_scores(x))

df = pd.concat([df.drop(['sentiments'], axis=1), df['sentiments'].apply(pd.Series)], axis=1)

df.head()

ID                                             Review  Rating  \
0   0  exceptional service nice all-around daughter s...       5   
1   1  beautiful relaxing jw marriott desert ridge re...       5   
2   2  great location great location 5 mins subway ta...       5   
3   3  pleased nice safe hotel, flower market hotel v...       3   
4   4  excellent hotel service great hotel excellent ...       4   

                                        review_clean    neg    neu    pos  \
0  exceptional service nice all-around daughter s...  0.046  0.691  0.263   
1  beautiful relax jw marriott desert ridge resor...  0.046  0.493  0.461   
2  great location great location mins subway take...  0.083  0.557  0.361   
3  pleased nice safe hotel flower market hotel va...  0.000  0.750  0.250   
4  excellent hotel service great hotel excellent ...  0.000  0.532  0.468   

   compound  
0    0.9877  
1    0.9970  
2    0.9808  
3    0.9281  
4    0.9718

In [15]:
# add number of characters columns
df["nb_chars"] = df["Review"].apply(lambda x: len(x))

# add number of words column
df["nb_words"] = df["Review"].apply(lambda x: len(x.split(" ")))

df.head()

ID                                             Review  Rating  \
0   0  exceptional service nice all-around daughter s...       5   
1   1  beautiful relaxing jw marriott desert ridge re...       5   
2   2  great location great location 5 mins subway ta...       5   
3   3  pleased nice safe hotel, flower market hotel v...       3   
4   4  excellent hotel service great hotel excellent ...       4   

                                        review_clean    neg    neu    pos  \
0  exceptional service nice all-around daughter s...  0.046  0.691  0.263   
1  beautiful relax jw marriott desert ridge resor...  0.046  0.493  0.461   
2  great location great location mins subway take...  0.083  0.557  0.361   
3  pleased nice safe hotel flower market hotel va...  0.000  0.750  0.250   
4  excellent hotel service great hotel excellent ...  0.000  0.532  0.468   

   compound  nb_chars  nb_words  
0    0.9877       937       134  
1    0.9970       791       108  
2    0.9808       473        73  
3    0.9281       342        51  
4    0.9718       244        33

In [16]:
# highest positive sentiment reviews (with more than 5 words)
df[df["nb_words"] >= 5].sort_values("pos", ascending = False)[["Review", "pos"]].head(10)

Review    pos
12780  perfect, impressive place, want clean comfy ro...  0.818
13758  hotel great hotel great money clean good restu...  0.799
10169  regular, amazing value, stay hotel let, great ...  0.796
10350  fantastic value money good value money, great ...  0.792
1517   great hotel loved hotel, location perfect staf...  0.787
10518  incredible value great price expedia package, ...  0.777
4919   best hotel, hotel gorgeous, beautiful clean sp...  0.763
6643   fabulous trip stayed just night wish longer, h...  0.757
3057   wonderful hotel perfect place stay excellent l...  0.756
10634  location superb, rooms good size appointed sta...  0.753

In [17]:
# lowest negative sentiment reviews (with more than 5 words)
df[df["nb_words"] >= 5].sort_values("neg", ascending = False)[["Review", "neg"]].head(10)

Review    neg
7808   bad hotel really awful place dirty room rude s...  0.646
13106  bad bad bad hotel verry bad pls not stay, clos...  0.645
6908   worn noisy worn noisy, remodel future, prices ...  0.527
12728  extremely disappointing experience disappointi...  0.507
10332  worst hotel room awful no windows poor lightin...  0.477
5671   eh, no didnt like room hotel super small does ...  0.461
9660   warwick bad good reviews warwick shocks staff ...  0.458
2677   food great rest service poor stayed hotel time...  0.440
12565  horrible stay bad staff rooms horrible staff u...  0.438
9486   ritz canal street terrible location equally ho...  0.438